## GeoCat experimental

In [28]:
import importlib
from collections import defaultdict
import numpy as np
import pandas as pd
import recommenders as rec
import geocat
import networkx as nx
import matplotlib.pyplot as plt
import areamanager
import math
import time
from IPython.core.debugger import set_trace
%matplotlib inline
def string_to_array(string):
    if string == '':
        return list()
    return eval(string)
importlib.reload(geocat)
importlib.reload(rec)
CITY="madison"



In [2]:
df_checkin=pd.read_csv("../data/checkin/"+CITY+".csv",converters={'categories':string_to_array})

df_checkin_train=pd.read_csv("../data/checkin/train/"+CITY+".csv",converters={'categories':string_to_array})


users_id=df_checkin['user_id'].drop_duplicates().reset_index(drop=True)
pois_id=df_checkin['business_id'].drop_duplicates().reset_index(drop=True)
user_num=len(users_id)
poi_num=len(pois_id)


df_checkin_train=df_checkin_train.set_index('user_id')

df_checkin_train.head()

,business_id,date,latitude,longitude,categories,subarea_id
user_id,,,,,,
0,1744,2017-12-15 14:05:01,43.075012,-89.447498,"[desserts, cafes, coffee]",2143.0
0,2746,2018-01-18 20:57:23,43.106774,-89.497738,"[desserts, tradamerican, diners, gastropubs, b...",2641.0
0,30,2018-01-23 23:14:37,43.072654,-89.384062,"[bars, mexican]",2133.0
0,432,2018-01-27 02:17:06,43.072439,-89.384610,"[tradamerican, bars]",2133.0
0,22,2018-01-27 02:31:26,43.075100,-89.381069,[mexican],2132.0


In [3]:
df_checkin_test=pd.read_csv("../data/checkin/test/"+CITY+".csv",converters={'categories':string_to_array})

In [4]:
ground_truth = defaultdict(set)
for index,row in df_checkin_test[['user_id','business_id']].drop_duplicates().iterrows():
    ground_truth[row['user_id']].add(row['business_id'])

In [5]:
poi_coos = {}
for lid,latlon in df_checkin[['business_id','latitude','longitude']].reset_index(drop=True).drop_duplicates().set_index('business_id').iterrows():
    poi_coos[lid] = tuple(latlon)

In [6]:
poi_cats = {}
for lid,row in df_checkin[['business_id','categories']].reset_index(drop=True).drop_duplicates(subset=['business_id']).set_index('business_id').iterrows():
    poi_cats[lid] = row.categories

In [7]:
dict_alias_title,category_tree,dict_alias_depth=geocat.category_manipulation_utils()
undirected_category_tree=category_tree.to_undirected()

In [8]:
training_matrix = np.zeros((user_num, poi_num))
for user_id,poi_id in df_checkin_train.business_id.iteritems():
   # print(user_id,poi_id)
    training_matrix[user_id,poi_id]+=1

In [9]:
all_uids = list(range(user_num))
all_lids = list(range(poi_num))

In [10]:
#np.random.shuffle(all_uids)

In [11]:
user_predict={}
for index,row in pd.read_csv("../data/result/sigir11_top_100.txt",header=None).iterrows():
    user_id=row[0]
    lid_list=row[1]
    user_predict[user_id]=eval(lid_list)

In [12]:
df_poi_neighbor=pd.read_csv("../data/neighbor/"+CITY+".csv",converters={'neighbors':string_to_array})
df_poi_neighbor=df_poi_neighbor.set_index("business_id")
poi_neighbors={}
for lid,row in df_poi_neighbor.iterrows():
    neighbors=row['neighbors']
    poi_neighbors[lid]=neighbors
df_poi_neighbor=None

In [13]:
import objfunction
importlib.reload(objfunction)

<module 'objfunction' from '/home/heitor/recsys/algorithms/objfunction.py'>

In [14]:
#lvisited=set(training_matrix[uid].nonzero()[0])

In [59]:
importlib.reload(rec)
importlib.reload(objfunction)
N=80
K=20
range_K=range(K)
overall_scores=np.zeros(poi_num)
for cnt, uid in enumerate(all_uids):
    if uid in ground_truth:
        
        print("userid →",uid)
        
        pois_score=rec.mostpopularnp(training_matrix,uid)
        tmp_rec_list=list(reversed(np.argsort(pois_score)))[0:N]
        tmp_score_list=list(reversed(np.sort(pois_score)))[0:N]
        #print(list(reversed(np.sort(pois_score)))[0:N])
        
        #print(tmp_rec_list)
        #break
        #tmp_rec_list=user_predict[uid][:N]
        rec_list=[]
        
        lids=training_matrix[uid].nonzero()[0]
        
        lid_visits=training_matrix[:,lids].sum(axis=0)
        mean_visits=lid_visits.mean()
        relevant_lids=lids[lid_visits>mean_visits]
        relevant_cats=set()
        for lid in relevant_lids:
            relevant_cats.update(poi_cats[lid])
        # log_size=training_matrix[0,training_matrix[0,:].nonzero()[0]].sum()
        user_log=training_matrix[uid]
        #print(mean_visits)
        log_poi_ids=list()
        poi_cover=list()
        for lid in user_log.nonzero()[0]:
            for visits in range(int(user_log[lid])):
                poi_cover.append(0)
                log_poi_ids.append(lid)
        log_size=len(log_poi_ids)
        assert user_log[user_log.nonzero()[0]].sum() == len(poi_cover)
#         print(uid)
#         print("Count:",cnt)
        div_geo_cat_weight = 0.75 # beta,this is here because of the work to be done on parameter customization for each user
        div_weight = 0.5 # lambda, geo vs cat
        current_proportionality=0
        final_scores=[]
        
        start_time = time.time()
        log_neighbors=dict()
        for poi_id in tmp_rec_list:
            neighbors=list()
            for id_neighbor in poi_neighbors[poi_id]:
                for i in range(log_size):
                    log_poi_id=log_poi_ids[i]
                    if log_poi_id == id_neighbor:
                        neighbors.append(i)
            log_neighbors[poi_id]=neighbors

        
        for i in range_K:
            #print(i)
            poi_to_insert=None
            max_objective_value=-200
            for j in range(len(tmp_rec_list)):
                candidate_poi_id=tmp_rec_list[j]
                candidate_score=tmp_score_list[j]
                ild_div=objfunction.min_dist_to_list_cat(candidate_poi_id,rec_list,poi_cats,undirected_category_tree)
                gc_div=objfunction.gc(candidate_poi_id,rec_list,relevant_cats,poi_cats)
                pr=objfunction.update_geo_cov(candidate_poi_id,log_poi_ids,K,poi_cover.copy(),poi_neighbors,log_neighbors[candidate_poi_id])
                
                objective_value=objfunction.ILD_GC_PR(candidate_score,ild_div,gc_div,pr,current_proportionality,K,div_geo_cat_weight,div_weight)
                #print(candidate_poi_id,ild_div,gc_div,max(0,pr-current_proportionality),objective_value)
                #print(candidate_poi_id,objective_value)

                if objective_value > max_objective_value:
                    max_objective_value=objective_value
                    poi_to_insert=candidate_poi_id
                pass
            if poi_to_insert is not None:
                #print(poi_to_insert,max_objective_value)
                
                rm_idx=tmp_rec_list.index(poi_to_insert)
                
                tmp_rec_list.pop(rm_idx)
                tmp_score_list.pop(rm_idx)
                rec_list.append(poi_to_insert)
                final_scores.append(max_objective_value)
                # remove from tmp_rec_list
                current_proportionality=objfunction.update_geo_cov(poi_to_insert,log_poi_ids,K,poi_cover,poi_neighbors,log_neighbors[poi_to_insert])
                #print(current_proportionality)
        print("time → %fs" % (time.time()-start_time))
#         print(final_scores)
#         print(rec_list)
        #print(list(reversed(np.sort(final_scores))))
        #print(np.array(rec_list)[list(reversed(np.argsort(final_scores)))])
        #print()
    

userid → 0
time → 4.431004s
userid → 1
time → 3.660097s
userid → 2
time → 4.151214s
userid → 3
time → 3.989806s
userid → 4
time → 4.707048s
userid → 5
time → 4.632485s
userid → 6
time → 4.451484s
userid → 7
time → 4.021662s
userid → 8
time → 4.730345s
userid → 9
time → 4.497933s
userid → 10
time → 4.452650s
userid → 11
time → 4.326638s
userid → 12
time → 4.235497s
userid → 13
time → 4.386007s
userid → 14
time → 4.590487s
userid → 15
time → 4.429693s
userid → 16
time → 4.331478s
userid → 17
time → 3.534001s
userid → 18
time → 3.893354s
userid → 19
time → 3.793668s
userid → 20
time → 4.316486s
userid → 21
time → 4.461761s
userid → 22
time → 3.944142s
userid → 23
time → 4.254403s
userid → 24
time → 4.528245s
userid → 25
time → 4.065346s
userid → 26
time → 4.078300s
userid → 27
time → 3.970846s
userid → 28
time → 4.341851s
userid → 29
time → 4.813803s
userid → 30
time → 4.232852s
userid → 31
time → 3.990122s
userid → 32
time → 4.079852s
userid → 33
time → 4.250236s
userid → 34
time → 4.380

KeyboardInterrupt: 